# FedPerf - MNIST + FedAvg algorithm

## Setup & Dependencies Installation

In [ ]:
%%capture
!pip install torchsummaryX unidecode

In [ ]:
%load_ext tensorboard

import copy
from functools import reduce
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import time
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.sampler import Sampler
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary
from torchsummaryX import summary as summaryx
from torchvision import transforms, utils, datasets
from tqdm.notebook import tqdm
from unidecode import unidecode

%matplotlib inline

# Check assigned GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

# set manual seed for reproducibility
RANDOM_SEED = 42

# general reproducibility
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed(RANDOM_SEED)

# gpu training specific
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## Mount GDrive

In [ ]:
BASE_DIR = '/content/drive/MyDrive/FedPerf/MNIST/FedAvg'

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    os.makedirs(BASE_DIR, exist_ok=True)
except:
    print("WARNING: Results won't be stored on GDrive")
    BASE_DIR = './'


## Load the MNIST Dataset

In [ ]:
# create transforms
# We will just convert to tensor and normalize since no special transforms are mentioned in the paper
transforms_mnist = transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,))
                                       ])

mnist_data_train = datasets.MNIST('./data/mnist/', train=True, download=True, transform=transforms_mnist)
mnist_data_test = datasets.MNIST('../data/mnist/', train=False, download=True, transform=transforms_mnist)

In [ ]:
classes = np.array(list(mnist_data_train.class_to_idx.values()))
classes_test = np.array(list(mnist_data_test.class_to_idx.values()))
num_classes = len(classes_test)
print("Classes: {} \tType: {}".format(classes, type(classes)))
print("Classes Test: {} \tType: {}".format(classes_test, type(classes)))


In [ ]:
print("Image Shape: {}".format(mnist_data_train.data[0].size()))

## Partitioning the Data (IID and non-IID)

In [ ]:
def iid_partition(dataset, clients):
  """
  I.I.D paritioning of data over clients
  Shuffle the data
  Split it between clients
  
  params:
    - dataset (torch.utils.Dataset): Dataset containing the CIFAR Images
    - clients (int): Number of Clients to split the data between

  returns:
    - Dictionary of image indexes for each client
  """

  num_items_per_client = int(len(dataset)/clients)
  client_dict = {}
  image_idxs = [i for i in range(len(dataset))]

  for i in range(clients):
    client_dict[i] = set(np.random.choice(image_idxs, num_items_per_client, replace=False))
    image_idxs = list(set(image_idxs) - client_dict[i])

  return client_dict

In [ ]:
def non_iid_partition(dataset, clients, total_shards, shards_size, num_shards_per_client):
  """
  non I.I.D parititioning of data over clients
  Sort the data by the digit label
  Divide the data into N shards of size S
  Each of the clients will get X shards

  params:
    - dataset (torch.utils.Dataset): Dataset containing the CIFAR Images
    - clients (int): Number of Clients to split the data between
    - total_shards (int): Number of shards to partition the data in
    - shards_size (int): Size of each shard 
    - num_shards_per_client (int): Number of shards of size shards_size that each client receives

  returns:
    - Dictionary of image indexes for each client
  """
  
  shard_idxs = [i for i in range(total_shards)]
  client_dict = {i: np.array([], dtype='int64') for i in range(clients)}
  idxs = np.arange(len(dataset))
  data_labels = np.array(dataset.targets)

  # sort the labels
  label_idxs = np.vstack((idxs, data_labels))
  label_idxs = label_idxs[:, label_idxs[1,:].argsort()]
  idxs = label_idxs[0,:]

  # divide the data into total_shards of size shards_size
  # assign num_shards_per_client to each client
  for i in range(clients):
    rand_set = set(np.random.choice(shard_idxs, num_shards_per_client, replace=False))
    shard_idxs = list(set(shard_idxs) - rand_set)

    for rand in rand_set:
      client_dict[i] = np.concatenate((client_dict[i], idxs[rand*shards_size:(rand+1)*shards_size]), axis=0)
  
  return client_dict





## Models

In [ ]:
class MNIST_2NN(nn.Module):
  """
  A simple multilayer-perceptron with 2-hidden layers with 200 units each
  using ReLu activations

  Total Expected Params: 199,210
  """
  def __init__(self):
    super(MNIST_2NN, self).__init__()

    self.fc1 = nn.Linear(28*28, 200)
    self.fc2 = nn.Linear(200, 200)
    self.fc3 = nn.Linear(200, 10)

  def forward(self, x):
    x = torch.flatten(x, 1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    out = self.fc3(x)

    return x

In [ ]:
class MNIST_CNN(nn.Module):
  """
  CNN with two 5x5 convolution lauers(the first with 32 channels, second with 64,
  each followed with 2x2 max pooling), a fully connected layer with 512 uunits and 
  ReLu activation, and the final Softmax output layer

  Total Expected Params: 1,663,370
  """
  def __init__(self):
    super(MNIST_CNN, self).__init__()

    self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
    self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
    
    self.pool = nn.MaxPool2d(2,2)
    self.dropout = nn.Dropout(p=0.2)

    self.fc1 = nn.Linear(1024, 512)
    self.out = nn.Linear(512, 10)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = self.dropout(x)
    x = torch.flatten(x, 1)
    x = F.relu(self.fc1(x))
    x = self.out(x)
    out = F.log_softmax(x, dim=1)

    return out

### Print Model Summaries

In [ ]:
mnist_mlp = MNIST_2NN()
mnist_cnn = MNIST_CNN()

if torch.cuda.is_available():
  mnist_mlp.cuda()
  mnist_cnn.cuda()

print("MNIST MLP SUMMARY")
print(summary(mnist_mlp, (28,28)))

print("\nMNIST CNN SUMMARY")
print(summary(mnist_cnn, (1, 28,28)))

## Plot utils

In [ ]:
def plot_scores(history, base_dir, title):
    accuracies = [x['accuracy'] for x in history]
    f1_macro = [x['f1_macro'] for x in history]
    f1_weighted = [x['f1_weighted'] for x in history]

    fig, ax = plt.subplots()
    ax.plot(accuracies, 'tab:orange')
    ax.set(xlabel='Rounds', ylabel='Test Accuracy', title=title)
    ax.grid()
    fig.savefig(f'{base_dir}/{title}_Test_Accuracy.jpg', format='jpg', dpi=300)
    plt.show()

    fig, ax = plt.subplots()
    ax.plot(f1_macro, 'tab:orange')
    ax.set(xlabel='Rounds', ylabel='Test F1 (macro)', title=title)
    ax.grid()
    fig.savefig(f'{base_dir}/{title}_Test_F1_Macro.jpg', format='jpg', dpi=300)
    plt.show()

    fig, ax = plt.subplots()
    ax.plot(f1_weighted, 'tab:orange')
    ax.set(xlabel='Rounds', ylabel='Test F1 (weighted)', title=title)
    ax.grid()
    fig.savefig(f'{base_dir}/{title}_Test_F1_Weighted.jpg', format='jpg', dpi=300)
    plt.show()


def plot_losses(history, base_dir, title):
    val_losses = [x['loss'] for x in history]
    train_losses = [x['train_loss'] for x in history]

    fig, ax = plt.subplots()
    ax.plot(train_losses, 'tab:orange')
    ax.set(xlabel='Rounds', ylabel='Train Loss', title=title)
    ax.grid()
    fig.savefig(f'{base_dir}/{title}_Train_Loss.jpg', format='jpg', dpi=300)
    plt.show()

    fig, ax = plt.subplots()
    ax.plot(val_losses, 'tab:orange')
    ax.set(xlabel='Rounds', ylabel='Test Loss', title=title)
    ax.grid()
    fig.savefig(f'{base_dir}/{title}_Test_Loss.jpg', format='jpg', dpi=300)
    plt.show()


## Federated Median

### Local Training (Client Update)

Local training for the model on client side

In [ ]:
class CustomDataset(Dataset):
  def __init__(self, dataset, idxs):
      self.dataset = dataset
      self.idxs = list(idxs)

  def __len__(self):
      return len(self.idxs)

  def __getitem__(self, item):
      image, label = self.dataset[self.idxs[item]]
      return image, label


class ClientUpdate(object):
  def __init__(self, dataset, batchSize, learning_rate, epochs, idxs):
    self.train_loader = DataLoader(CustomDataset(dataset, idxs), batch_size=batchSize, shuffle=True)

    self.learning_rate = learning_rate
    self.epochs = epochs

  def train(self, model):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=self.learning_rate, momentum=0.5)
    # optimizer = torch.optim.Adam(model.parameters(), lr=self.learning_rate)

    e_loss = []
    for epoch in range(1, self.epochs+1):

      train_loss = 0.0
      model.train()
      for data, labels in self.train_loader:

        if torch.cuda.is_available():
          data, labels = data.cuda(), labels.cuda()

        # clear the gradients
        optimizer.zero_grad()
        # make a forward pass
        output = model(data)
        # calculate the loss
        loss = criterion(output, labels)
        # do a backwards pass
        loss.backward()
        # perform a single optimization step
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)

      # average losses
      train_loss = train_loss/len(self.train_loader.dataset)
      e_loss.append(train_loss)

    total_loss = sum(e_loss)/len(e_loss)

    return model.state_dict(), total_loss

### Server Side Training

In [ ]:
def training(model, rounds, batch_size, lr, ds, ds_test, data_dict, C, K, E, M, plt_title, plt_color, test_history=[], logdir='.'):
  """
  Function implements the Federated Averaging Algorithm from the FedAvg paper.
  Specifically, this function is used for the server side training and weight update

  Params:
    - model:           PyTorch model to train
    - rounds:          Number of communication rounds for the client update
    - batch_size:      Batch size for client update training
    - lr:              Learning rate used for client update training
    - ds:              Dataset used for training
    - ds_test:         Dataset used for testing
    - data_dict:       Type of data partition used for training (IID or non-IID)
    - C:               Fraction of clients randomly chosen to perform computation on each round
    - K:               Total number of clients
    - E:               Number of training passes each client makes over its local dataset per round
    - tb_writer_name:  Directory name to save the tensorboard logs
  Returns:
    - model:           Trained model on the server
  """

  # global model weights
  global_weights = model.state_dict()
  num_clients = K
  pk = np.ones(num_clients) * 1.0 / num_clients

  # training loss
  # train_accuracy = []
  train_loss = []
  test_accuracy = []
  test_loss = []


  # measure time
  start = time.time()

  for curr_round in range(1, rounds+1):
    w, local_loss = [], []

    m = max(int(C*K), 1)
    # newM = max(int(M*m), 1)
    
    # c = 0
    S_t = np.random.choice(range(K), m, replace=False)
    print('Clients: {}/{} -> {}'.format(len(S_t), K, S_t))

    # print('Robustness..')
    # for i in range(len(S_t)):
    #   if c == newM:
    #     break
    #   c += 1

    #   local_update = ClientUpdate(dataset=ds, batchSize=batch_size, learning_rate=lr, epochs=E, idxs=data_dict[S_t[i]])
    #   weights, loss = local_update.train(model=copy.deepcopy(model))
            
    #   for k in weights.keys():
    #     t = torch.Tensor(weights[k].shape).cuda()
    #     t.fill_(0.1)
    #     weights[k] = t     

    #   w.append(copy.deepcopy(weights))
    #   local_loss.append(copy.deepcopy(loss))

    # for k in range(newM, len(S_t)):
    for k in tqdm(range(len(S_t))):
      local_update = ClientUpdate(dataset=ds, batchSize=batch_size, learning_rate=lr, epochs=E, idxs=data_dict[S_t[k]])
      weights, loss = local_update.train(model=copy.deepcopy(model))
      
      w.append(copy.deepcopy(weights))
      local_loss.append(copy.deepcopy(loss))

    target = copy.deepcopy(w[0]);
    weights_med = copy.deepcopy(w[0]);
    for k in weights_med.keys():
      tmp = copy.deepcopy(torch.median(torch.stack([w[i][k].data for i in range(0, len(w))]), dim=0))[0]
      
      target[k].data = tmp    

    global_weights = target
    
    # move the updated weights to our model state dict
    model.load_state_dict(global_weights)

    # loss
    loss_avg = sum(local_loss) / len(local_loss)
    print('Round: {}... \tAverage Loss: {}'.format(curr_round, round(loss_avg, 3)))
    train_loss.append(loss_avg)

    # test
    test_criterion = nn.CrossEntropyLoss()
    # test_accuracy_current, test_loss_current =  testing(copy.deepcopy(model), ds_test, 128, test_criterion, num_classes, classes_test)
    # test_accuracy.append(test_accuracy_current)
    # test_loss.append(test_loss_current)    
    scores =  testing(copy.deepcopy(model), ds_test, 128, test_criterion, num_classes, classes_test)
    test_accuracy.append(scores['accuracy'])
    test_loss.append(scores['loss'])
    scores['train_loss'] = loss_avg
    test_history.append(scores)

  end = time.time()
  
  fig, ax = plt.subplots()
  x_axis = np.arange(1, rounds+1)
  y_axis = np.array(train_loss)
  ax.plot(x_axis, y_axis, 'tab:'+plt_color)
  ax.set(xlabel='Number of Rounds', ylabel='Train Loss',
       title=plt_title)
  ax.grid()
  # fig.savefig(f'{logdir}/{plt_title}_Train_loss.jpg', format='jpg', dpi=300)
  plt.show()

  fig, ax = plt.subplots()
  x_axis = np.arange(1, rounds+1)
  y_axis = np.array(test_loss)
  ax.plot(x_axis, y_axis, 'tab:'+plt_color)
  ax.set(xlabel='Number of Rounds', ylabel='Test Loss',
       title=plt_title)
  ax.grid()
  # fig.savefig(f'{logdir}/{plt_title}_Test_loss.jpg', format='jpg', dpi=300)
  plt.show()

  fig, ax = plt.subplots()
  x_axis = np.arange(1, rounds+1)
  y_axis = np.array(test_accuracy)
  ax.plot(x_axis, y_axis, 'tab:'+plt_color)
  ax.set(xlabel='Number of Rounds', ylabel='Test Accuracy',
       title=plt_title)
  ax.grid()
  # fig.savefig(f'{logdir}/{plt_title}_Test_Accuracy.jpg', format='jpg', dpi=300)
  plt.show()
  
  print("Training Done!")
  print("Total time taken to Train: {}\n\n".format(end-start))
  
  # return model, train_loss, test_accuracy, test_loss
  return model, test_history

## Testing Loop

In [ ]:
def testing(model, dataset, bs, criterion, num_classes, classes, print_info=False):
  #test loss 
  test_loss = 0.0
  correct_class = list(0. for i in range(num_classes))
  total_class = list(0. for i in range(num_classes))

  test_loader = DataLoader(dataset, batch_size=bs)
  l = len(test_loader)
  model.eval()
  # for data, labels in test_loader:
  for i, (data, labels) in enumerate(tqdm(test_loader)):

    if torch.cuda.is_available():
      data, labels = data.cuda(), labels.cuda()

    output = model(data)
    loss = criterion(output, labels)
    test_loss += loss.item()*data.size(0)

    _, pred = torch.max(output, 1)

    # For F1Score
    y_true = np.append(y_true, labels.data.view_as(pred).cpu().numpy()) if i != 0 else labels.data.view_as(pred).cpu().numpy()
    y_hat = np.append(y_hat, pred.cpu().numpy()) if i != 0 else pred.cpu().numpy()

    correct_tensor = pred.eq(labels.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not torch.cuda.is_available() else np.squeeze(correct_tensor.cpu().numpy())

    #test accuracy for each object class
    for i in range(num_classes):
      label = labels.data[i]
      correct_class[label] += correct[i].item()
      total_class[label] += 1
    
  # avg test loss
  test_loss = test_loss/len(test_loader.dataset)
  test_accuracy = 100. * np.sum(correct_class) / np.sum(total_class)
  print("Test Loss: {:.6f}\n".format(test_loss))

  # Avg F1 Score
  f1_macro = f1_score(y_true, y_hat, average='macro')
  # F1-Score -> weigthed to consider class imbalance
  f1_weighted =  f1_score(y_true, y_hat, average='weighted')
  print("F1 Score: {:.6f} (macro) {:.6f} (weighted) %\n".format(f1_macro, f1_weighted))

  if print_info:
    for i in range(num_classes):
      if total_class[i]>0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % 
              (classes[i], 100 * correct_class[i] / total_class[i],
              np.sum(correct_class[i]), np.sum(total_class[i])))
      else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

  print('\nFinal Test  Accuracy: {:.3f} ({}/{})'.format(
        100. * np.sum(correct_class) / np.sum(total_class),
        np.sum(correct_class), np.sum(total_class)))
  
  return {'loss': test_loss, 'accuracy': test_accuracy, 'f1_macro': f1_macro, 'f1_weighted': f1_weighted}

## Experiments

In [ ]:
class Hyperparameters():

    def __init__(self, total_clients):
        # number of training rounds
        self.rounds = 50
        # client fraction
        self.C = 0.3
        # number of clients
        self.K = total_clients
        # number of training passes on local dataset for each roung
        self.E = 5
        # FedMed param
        self.M = 0.01
        # batch size
        self.batch_size = 10
        # learning Rate
        self.lr = 0.05

In [ ]:
hparams = Hyperparameters(total_clients=100)
hparams.__dict__

In [ ]:
# Sweeping parameter
PARAM_NAME = 'clients_fraction'
PARAM_VALUE = hparams.C
exp_id = f'{PARAM_NAME}/{PARAM_VALUE}'
exp_id

In [ ]:
EXP_DIR = f'{BASE_DIR}/{exp_id}'
os.makedirs(EXP_DIR, exist_ok=True)

# tb_logger = SummaryWriter(log_dir)
# print(f'TBoard logger created at: {log_dir}')
EXP_DIR

### MNIST CNN on IID

In [ ]:
exp_log = dict()

In [ ]:
title = 'MNIST CNN on IID Dataset'

In [ ]:
# train_ds, data_dict, test_ds = iid_partition(corpus, seq_length, val_split=True)
iid_dict = iid_partition(mnist_data_train, hparams.K)

total_clients = len(iid_dict.keys())
'Total users:', total_clients

In [ ]:
def run_experiment(run_id):

    mnist_cnn = MNIST_CNN()

    if torch.cuda.is_available():
        mnist_cnn.cuda()
    
    test_history = []

    mnist_cnn_iid_trained, test_history = training(mnist_cnn,
                                                 hparams.rounds, hparams.batch_size, hparams.lr,
                                                 mnist_data_train, mnist_data_test, iid_dict,
                                                 hparams.C, hparams.K, hparams.E, hparams.M,
                                                 title, "orange",
                                                 test_history
                                                 )
    
    final_scores = testing(mnist_cnn_iid_trained, mnist_data_test, hparams.batch_size * 2, nn.CrossEntropyLoss(), num_classes, classes_test)
    print(f'\n\n========================================================\n\n')
    print(f'Final scores for Exp {run_id} \n {final_scores}')

    log = {
        'history': test_history,
        'hyperparams': hparams.__dict__
    }

    with open(f'{EXP_DIR}/results_iid_cnn_{run_id}.pkl', 'wb') as file:
        pickle.dump(log, file)

    return test_history
    

In [ ]:
exp_history = list()
for run_id in range(2):  # TOTAL RUNS
    print(f'============== RUNNING EXPERIMENT #{run_id} ==============')
    exp_history.append(run_experiment(run_id))
    print(f'\n\n========================================================\n\n')

In [ ]:
exp_log[title] = {
    'history': exp_history,
    'hyperparams': hparams.__dict__
}

In [ ]:
df = None
for i, e in enumerate(exp_history):
    if i == 0:
        df = pd.json_normalize(e)
        continue
    df = df + pd.json_normalize(e)
    
df_avg = df / len(exp_history)
avg_history = df_avg.to_dict(orient='records')

In [ ]:
plot_scores(history=avg_history, base_dir=EXP_DIR, title=title)

In [ ]:
plot_losses(history=avg_history, base_dir=EXP_DIR, title=title)

### MNIST CNN Non-IID

In [ ]:
exp_log = dict()

In [ ]:
title = 'MNIST CNN on Non-IID Dataset'

In [ ]:
# Skewness
# 100, 200, 250, 2 - Low
# 100, 500, 120, 5 - Medium
# 100, 1000, 60, 10 - High
clients = hparams.K
num_shards_per_client = 2
total_shards = 200
shards_size = 250

hparams.shards = num_shards_per_client

clients, total_shards, shards_size, num_shards_per_client

In [ ]:
niid_dict = non_iid_partition(dataset=mnist_data_train,
                                clients=hparams.K,
                                total_shards=total_shards,
                                shards_size=shards_size,
                                num_shards_per_client=num_shards_per_client)

total_clients = len(niid_dict.keys())
'Total users:', total_clients

In [ ]:
def run_experiment(run_id):

    mnist_cnn = MNIST_CNN()

    if torch.cuda.is_available():
        mnist_cnn.cuda()
    
    test_history = []
    
    mnist_cnn_niid_trained, test_history = training(mnist_cnn,
                                                 hparams.rounds, hparams.batch_size, hparams.lr,
                                                 mnist_data_train, mnist_data_test, iid_dict,
                                                 hparams.C, hparams.K, hparams.E, hparams.M,
                                                 title, "orange",
                                                 test_history
                                                 )
    
    final_scores = testing(mnist_cnn_niid_trained, mnist_data_test, hparams.batch_size * 2, nn.CrossEntropyLoss(), num_classes, classes_test)
    print(f'\n\n========================================================\n\n')
    print(f'Final scores for Exp {run_id} \n {final_scores}')

    log = {
        'history': test_history,
        'hyperparams': hparams.__dict__
    }

    with open(f'{EXP_DIR}/results_niid_cnn_{run_id}.pkl', 'wb') as file:
        pickle.dump(log, file)

    return test_history

    

In [ ]:
exp_history = list()
for run_id in range(2):  # TOTAL RUNS
    print(f'============== RUNNING EXPERIMENT #{run_id} ==============')
    exp_history.append(run_experiment(run_id))
    print(f'\n\n========================================================\n\n')

In [ ]:
exp_log[title] = {
    'history': exp_history,
    'hyperparams': hparams.__dict__
}

In [ ]:
df = None
for i, e in enumerate(exp_history):
    if i == 0:
        df = pd.json_normalize(e)
        continue
    df = df + pd.json_normalize(e)
    
df_avg = df / len(exp_history)
avg_history = df_avg.to_dict(orient='records')

In [ ]:
plot_scores(history=avg_history, base_dir=EXP_DIR, title=title)

In [ ]:
plot_losses(history=avg_history, base_dir=EXP_DIR, title=title)

### MNIST MLP on IID

In [ ]:
exp_log = dict()

In [ ]:
title = 'MNIST MLP on IID Dataset'

In [ ]:
# train_ds, data_dict, test_ds = iid_partition(corpus, seq_length, val_split=True)
iid_dict = iid_partition(mnist_data_train, hparams.K)

total_clients = len(iid_dict.keys())
'Total users:', total_clients

In [ ]:
def run_experiment(run_id):

    mnist_mlp = MNIST_2NN()

    if torch.cuda.is_available():
        mnist_mlp.cuda()
    
    test_history = []
    
    mnist_mlp_iid_trained, test_history = training(mnist_mlp,
                                                 hparams.rounds, hparams.batch_size, hparams.lr,
                                                 mnist_data_train, mnist_data_test, iid_dict,
                                                 hparams.C, hparams.K, hparams.E, hparams.M,
                                                 title, "orange",
                                                 test_history
                                                 )
    
    final_scores = testing(mnist_mlp_iid_trained, mnist_data_test, hparams.batch_size * 2, nn.CrossEntropyLoss(), num_classes, classes_test)
    print(f'\n\n========================================================\n\n')
    print(f'Final scores for Exp {run_id} \n {final_scores}')

    log = {
        'history': test_history,
        'hyperparams': hparams.__dict__
    }

    with open(f'{EXP_DIR}/results_iid_mlp_{run_id}.pkl', 'wb') as file:
        pickle.dump(log, file)

    return test_history
    

In [ ]:
exp_history = list()
for run_id in range(2):  # TOTAL RUNS
    print(f'============== RUNNING EXPERIMENT #{run_id} ==============')
    exp_history.append(run_experiment(run_id))
    print(f'\n\n========================================================\n\n')

In [ ]:
exp_log[title] = {
    'history': exp_history,
    'hyperparams': hparams.__dict__
}

In [ ]:
df = None
for i, e in enumerate(exp_history):
    if i == 0:
        df = pd.json_normalize(e)
        continue
    df = df + pd.json_normalize(e)
    
df_avg = df / len(exp_history)
avg_history = df_avg.to_dict(orient='records')

In [ ]:
plot_scores(history=avg_history, base_dir=EXP_DIR, title=title)

In [ ]:
plot_losses(history=avg_history, base_dir=EXP_DIR, title=title)

### MNIST CNN Non-IID

In [ ]:
exp_log = dict()

In [ ]:
title = 'MNIST MLP on Non-IID Dataset'

In [ ]:
# Skewness
# 100, 200, 250, 2 - Low
# 100, 500, 120, 5 - Medium
# 100, 1000, 60, 10 - High
clients = hparams.K
num_shards_per_client = 2
total_shards = 200
shards_size = 250

hparams.shards = num_shards_per_client

clients, total_shards, shards_size, num_shards_per_client

In [ ]:
niid_dict = non_iid_partition(dataset=mnist_data_train,
                                clients=hparams.K,
                                total_shards=total_shards,
                                shards_size=shards_size,
                                num_shards_per_client=num_shards_per_client)

total_clients = len(niid_dict.keys())
'Total users:', total_clients

In [ ]:
def run_experiment(run_id):

    mnist_mlp = MNIST_2NN()

    if torch.cuda.is_available():
        mnist_mlp.cuda()
    
    test_history = []
    
    mnist_mlp_niid_trained, test_history = training(mnist_mlp,
                                                 hparams.rounds, hparams.batch_size, hparams.lr,
                                                 mnist_data_train, mnist_data_test, iid_dict,
                                                 hparams.C, hparams.K, hparams.E, hparams.M,
                                                 title, "orange",
                                                 test_history
                                                 )
    
    final_scores = testing(mnist_mlp_niid_trained, mnist_data_test, hparams.batch_size * 2, nn.CrossEntropyLoss(), num_classes, classes_test)
    print(f'\n\n========================================================\n\n')
    print(f'Final scores for Exp {run_id} \n {final_scores}')

    log = {
        'history': test_history,
        'hyperparams': hparams.__dict__
    }

    with open(f'{EXP_DIR}/results_niid_mlp_{run_id}.pkl', 'wb') as file:
        pickle.dump(log, file)

    return test_history

    

In [ ]:
exp_history = list()
for run_id in range(2):  # TOTAL RUNS
    print(f'============== RUNNING EXPERIMENT #{run_id} ==============')
    exp_history.append(run_experiment(run_id))
    print(f'\n\n========================================================\n\n')

In [ ]:
exp_log[title] = {
    'history': exp_history,
    'hyperparams': hparams.__dict__
}

In [ ]:
df = None
for i, e in enumerate(exp_history):
    if i == 0:
        df = pd.json_normalize(e)
        continue
    df = df + pd.json_normalize(e)
    
df_avg = df / len(exp_history)
avg_history = df_avg.to_dict(orient='records')

In [ ]:
plot_scores(history=avg_history, base_dir=EXP_DIR, title=title)

In [ ]:
plot_losses(history=avg_history, base_dir=EXP_DIR, title=title)